## WeatherPy 
### Generate 500+ random latitudes and longitudes and use Weathermap.org and API keys to retrieve and graph weather day showing various relationships. 

In [12]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from citipy import citipy
from random import uniform
from config import api_key

In [25]:
# Create a df to lat, lon 
column_names = ["City", "Country"]
cities_df = pd.DataFrame(columns=column_names)
cities_df

,City,Country


## Generate Cites List from Lats and Longs

In [26]:
# Randomly generate a list of 1500 cities. Use 50 to test for duplicates.
for x in range(50):
    lat, lng = np.random.uniform(low=-90.000, high=90.000), np.random.uniform(low=-180.000, high=180.000)
    city = citipy.nearest_city(lat, lng)
    cities_df = cities_df.append({"City": city.city_name,"Country": city.country_code,}, ignore_index=True)
cities_df.head()

,City,Country
0,jiamusi,cn
1,yashkul,ru
2,nanortalik,gl
3,san ignacio,bo
4,port alfred,za


In [27]:
#Remove duplicate cities
cities_df = cities_df.drop_duplicates(subset='City').reset_index()
cities_df.count()

index      42
City       42
Country    42
dtype: int64

## Perform API Calls

In [28]:
#Create a df to add all the data from the API calls.
column_names = ["City", "Country","Temperature (F)",
                "Latitude","Longitude","Humidity (%)",
                "Cloudiness (%)", "Wind Speed (mph)"]
clean_cities_df = pd.DataFrame(columns=column_names)
clean_cities_df.head()

,City,Country,Temperature (F),Latitude,Longitude,Humidity (%),Cloudiness (%),Wind Speed (mph)


In [29]:
# Loop through the list of cities and add the data to the df.
url = "http://api.openweathermap.org/data/2.5/weather?"

row_count = 1

print("Beginning Data Retrieval")
for index, row in cities_df.iterrows():
    target_city = row["City"]
    target_url = url + "appid=" + api_key + "&units=IMPERIAL" + "&q=" + target_city.replace(" ","")
    city_data = requests.get(target_url).json()
    if city_data["cod"] == "404":
        print("Proceesing: City # " , row_count, ' | '
              ,cities_df["City"][row_count-1], cities_df["Country"][row_count-1]," not found.")
        row_count += 1

    else:
        clean_cities_df.set_value(index, "City", city_data["name"])
        clean_cities_df.set_value(index, "Country", city_data["sys"]["country"])
        clean_cities_df.set_value(index, "Temperature (F)", city_data["main"]["temp"])
        clean_cities_df.set_value(index, "Latitude", city_data["coord"]["lat"])
        clean_cities_df.set_value(index, "Longitude", city_data["coord"]["lon"])
        clean_cities_df.set_value(index, "Humidity (%)", city_data["main"]["humidity"])
        clean_cities_df.set_value(index, "Cloudiness (%)", city_data["clouds"]["all"])
        clean_cities_df.set_value(index, "Wind Speed (mph)", city_data["wind"]["speed"])
        
        print("------------------------")
        print("Proceesing: City # " , row_count, ' | ' , city_data["name"], city_data["sys"]["country"])
        print(target_url)
        row_count += 1

Beginning Data Retrieval
------------------------
Proceesing: City #  1  |  Jiamusi CN
http://api.openweathermap.org/data/2.5/weather?appid=72507afc9de717437b6555b2efaba1b1&units=IMPERIAL&q=jiamusi
------------------------
Proceesing: City #  2  |  Yashkul RU
http://api.openweathermap.org/data/2.5/weather?appid=72507afc9de717437b6555b2efaba1b1&units=IMPERIAL&q=yashkul
------------------------
Proceesing: City #  3  |  Nanortalik GL
http://api.openweathermap.org/data/2.5/weather?appid=72507afc9de717437b6555b2efaba1b1&units=IMPERIAL&q=nanortalik
Proceesing: City #  4  |  san ignacio bo  not found.
Proceesing: City #  5  |  port alfred za  not found.
Proceesing: City #  6  |  hervey bay au  not found.
------------------------
Proceesing: City #  7  |  Atuona PF
http://api.openweathermap.org/data/2.5/weather?appid=72507afc9de717437b6555b2efaba1b1&units=IMPERIAL&q=atuona
------------------------
Proceesing: City #  8  |  Noumea NC
http://api.openweathermap.org/data/2.5/weather?appid=72507af

In [30]:
clean_cities_df

,City,Country,Temperature (F),Latitude,Longitude,Humidity (%),Cloudiness (%),Wind Speed (mph)
0,Jiamusi,CN,-6.33,46.83,130.35,73,56,2.95
1,Yashkul,RU,31.97,46.17,45.34,85,0,2.95
2,Nanortalik,GL,23.87,60.14,-45.24,100,36,7.2
6,Atuona,PF,78.59,-9.8,-139.03,100,88,18.28
7,Noumea,NC,66.2,-22.28,166.46,82,75,9.33
8,Kaeo,NZ,58.48,-35.1,173.78,85,92,17.83
9,Belgrade,RS,48.2,44.8,20.47,81,75,5.82
11,Busselton,AU,63.02,-33.65,115.33,100,12,2.28
12,Yenagoa,NG,86.29,4.92,6.26,69,20,10.56
14,Rikitea,PF,71.89,-23.12,-134.97,100,92,10.89


In [32]:
# Save data to a CSV file
clean_cities_df.to_csv("City_Data.csv")